In [1]:
from deeptables.models import deeptable,deepnets
from deeptables.utils import consts,dt_logging,batch_trainer
from deeptables.datasets import dsutils
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd
import numpy as np

/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
data = dsutils.load_adult()

Base dir:/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/deeptables/datasets


In [3]:
conf1 = deeptable.ModelConfig(
    name='conf1',
    apply_gbm_features=True,
    gbm_feature_type='dense',
    auto_discrete=True,
    metrics=['AUC'],
)

conf2 = deeptable.ModelConfig(
    name='conf2',
    fixed_embedding_dim=False,
    embeddings_output_dim=0,
    apply_gbm_features=False, #*
    gbm_feature_type='dense',
    auto_discrete=False, #*
    metrics=['AUC'],

)
bt = batch_trainer.BatchTrainer(data,'x_14',
                                eval_size=0.2,
                                validation_size=0.2,
                                eval_metrics=['AUC','accuracy','recall','precision','f1'], #auc/recall/precision/f1/mse/mae/msle/rmse/r2
                                verbose=1,
                                dt_epochs=15,
                                dt_config=[conf1,conf2],
                                dt_nets=[['dnn_nets'],['cross_nets']],
                               )
ms = bt.start()

2 class detected, {' <=50K', ' >50K'}, so inferred as a [binary classification] task
Start training DT model.['dnn_nets']
train metrics:['AUC']
eval metrics:['AUC', 'accuracy', 'recall', 'precision', 'f1']
Fitting model...
X_train.shape:(26048, 14),y_train.shape:(26048,)
2 class detected, {' <=50K', ' >50K'}, so inferred as a [binary classification] task
Preparing features cost:0.019079923629760742
Imputation cost:0.08739614486694336
Categorical encoding cost:0.14487409591674805
Discretization cost:0.08411478996276855
Extracting gbm features cost:1.2587871551513672
fit_transform cost:1.6632287502288818
Injected a callback [EarlyStopping]. monitor:val_AUC, patience:1, mode:max
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (14)', 'input_continuous_all: (6)', 'input_continuous_gbm_leaves: (100)']
----------------------------------

In [4]:
ms.leaderboard()

,model,type,*auc,accuracy,recall,precision,f1
0,conf1 - ['dnn_nets'] - eval,eval,0.912038,0.860587,0.582898,0.768503,0.662955
1,conf1 - ['cross_nets'] - eval,eval,0.911162,0.860894,0.603133,0.756137,0.671024
2,conf2 - ['cross_nets'] - eval,eval,0.905784,0.852756,0.565274,0.747196,0.643627
3,conf2 - ['dnn_nets'] - eval,eval,0.903408,0.851681,0.560705,0.745660,0.640089
